In [1]:
import pandas as pd
import spacy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
en = spacy.load('en_core_web_sm')
STOPWORDS = en.Defaults.stop_words

noticias = pd.read_json('../model/News_Category_Dataset_v3.json', lines=True)
df = noticias.drop(columns=['authors','link','date'])
df

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...
...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr..."
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked..."
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...


In [3]:
#recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')


# Pré-processamento de texto em inglês
stopwords = set(nltk.corpus.stopwords.words('english'))

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords]
    return " ".join(tokens)


[nltk_data] Downloading package punkt to /home/joaopedro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joaopedro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df['text'] = df['headline'] + " " + df['short_description']
df['text'] = df['text'].apply(preprocess_text)

In [5]:
# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])

In [6]:
# Codificação das categorias
category_mapping = {category: i for i, category in enumerate(df['category'])}
y = df['category'].map(category_mapping)



In [7]:
# Separação em dados de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
# Predição e avaliação do modelo
y_pred = knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo:", accuracy)

In [ ]:
print("Relatório de classificação:")
print(classification_report(y_test, y_pred))